In [ ]:
import numpy as np
from numpy.random import random
from copy import  deepcopy

# Load input


In [ ]:
from envyron.io.input import Input

In [ ]:
natoms = 3
my_input = Input(natoms=natoms, filename='input.yml')
my_input.ions

# Initialize a cell


In [ ]:
from envyron.domains.cell import EnvironGrid

In [ ]:
L = 20.

at = np.array([
    [L, 0., 0.],
    [0., L, 0.],
    [0., 0., L],
])

nr = np.array([80, 80, 80])

grid = EnvironGrid(at, nr, label='system')
print(grid.origin)
print(grid.lattice)
print(grid.volume)

# Compute minimum image distances


In [ ]:
r, r2 = grid.get_min_distance(grid.origin)

# Initialize a density


In [ ]:
from envyron.representations import EnvironDensity

In [ ]:
density = EnvironDensity(grid, data=random(nr), label='electrons')
print(density.label)
# density.standard_view()

# Compute multipoles


In [ ]:
origin = np.ones(3) * 0.5
density.compute_multipoles(origin)
print(f"charge = {density.charge}")
print(f"dipole = {density.dipole}")
print(f"quadrupole = {density.quadrupole}")

# Compute euclidean norm


In [ ]:
en = density.euclidean_norm()
# en

# Compute quadratic mean


In [ ]:
qm = density.quadratic_mean()
# qm

# Compute scalar product


In [ ]:
other_density = EnvironDensity(grid, data=random(nr))
sp = density.scalar_product(other_density)
# sp

# Gradient

In [ ]:
from envyron.representations import EnvironGradient

# Compute gradient (FFT - DFTPY)

In [ ]:
gradient = EnvironGradient(grid, data=density.gradient())
# gradient.standard_view()

# Compute gradient (FFT - Environ)

In [ ]:
from envyron.cores import FFTCore

In [ ]:
fft_core = FFTCore(grid)
gradient = fft_core.gradient(density)
# gradient.standard_view()

# Initialize a gradient


In [ ]:
gradient = EnvironGradient(grid, data=random((3, *nr)), label='electrons')
print(gradient.label)
# gradient.standard_view()

# Compute modulus


In [ ]:
mod = gradient.modulus
print(mod.label)
# mod.standard_view()

# Compute scalar products


In [ ]:
other_gradient = EnvironGradient(grid, data=random((3, *nr)))
sgp = gradient.scalar_product(other_gradient)
# sgp.standard_view()

In [ ]:
sdp = gradient.scalar_product(density)
# sdp

# Initialize a hessian


In [ ]:
from envyron.representations import EnvironHessian

In [ ]:
hessian = EnvironHessian(grid, data=random((9, *nr)), label='boundary')
print(hessian.label)
# hessian.standard_view()

# Compute laplacian


In [ ]:
lapl = hessian.trace
print(lapl.label)
# hessian.laplacian.standard_view()

# Compute scalar product


In [ ]:
sgp = hessian.scalar_gradient_product(gradient)
# sgp.standard_view()

# Initialize a gaussian function


In [ ]:
from envyron.representations.functions import EnvironGaussian

In [ ]:
gaussian = EnvironGaussian(
    grid,
    kind=1,
    dim=0,
    axis=0,
    width=0.5,
    spread=0.5,
    volume=0.5,
    pos=np.array([5, 5, 5]),
    label='ions',
)
gaussian.kind

# Compute density of function


In [ ]:
gd = gaussian.density
print(gd.label)
print(gd.shape)
print(gd.size)
print(gd[gd != 0].size)

# Compute gradient of function


In [ ]:
gg = gaussian.gradient
print(gg.label)
print(gg.shape)
print(gg.size)
print(gg[gg != 0].size)

# Initialize an erfc function


In [ ]:
from envyron.representations.functions import EnvironERFC

In [ ]:
erfc = EnvironERFC(
    grid,
    kind=2,
    dim=0,
    axis=0,
    width=0.5,
    spread=0.5,
    volume=0.5,
    pos=np.array([5, 5, 5]),
    label='soft_spheres',
)
erfc.kind

# Compute density of function


In [ ]:
ed = erfc.density
print(ed.label)
print(ed.shape)
print(ed.size)
print(ed[ed != 0].size)

# Compute gradient of function


In [ ]:
eg = erfc.gradient
print(eg.label)
print(eg.shape)
print(eg.size)
print(eg[eg != 0].size)

# Compute laplacian of function


In [ ]:
el = erfc.laplacian
print(el.label)
print(el.shape)
print(el.size)
print(el[el != 0].size)

# Compute hessian of function


In [ ]:
eh = erfc.hessian
print(eh.label)
print(eh.shape)
print(eh.size)
print(eh[eh != 0].size)

# Compute derivative of function


In [ ]:
edv = erfc.derivative
print(edv.label)
print(edv.shape)
print(edv.size)
print(edv[edv != 0].size)

# Initialize ions

In [ ]:
from envyron.physical import EnvironIons

In [ ]:
ions = EnvironIons(
    nions=natoms,
    ntypes=2,
    itypes=[0, 1, 1],
    ion_ids=['O', 'H'],
    zv=[6.0, 1.0],
    atomicspread=my_input.ions.atomicspread,
    corespread=my_input.ions.corespread,
    solvationrad=my_input.ions.solvationrad,
    radius_mode=my_input.solvent.radius_mode,
    is_soft_cavity=False,
    smear=True,
    fill_cores=True,
    grid=grid,
)

In [ ]:
coords = np.array([
    [11.79, 12.05, 11.50],
    [13.45, 11.22, 11.50],
    [10.56, 10.66, 11.50],
])

In [ ]:
for ion in ions.smeared_ions:
    print(
        ion.label,
        ion.kind,
        ion.dim,
        ion.axis,
        ion.width,
        ion.spread,
        ion.volume,
        ion.pos,
    )

In [ ]:
ions.smeared_ions[0].volume

In [ ]:
ions.iontypes[1].solvationrad

In [ ]:
ions.update(coords)

In [ ]:
for ion in ions.smeared_ions:
    print(ion.pos)

In [ ]:
print(f"charge     = {ions.charge}")
print(f"center     = {ions.com}")
print(f"quad corr  = {ions.quadrupole_correction}")
print(f"quad pc    = {ions.quadrupole_pc}")
print(f"quad gauss = {ions.quadrupole_gauss}")
print(f"pot shift  = {ions.potential_shift}")

In [ ]:
d = ions.density

In [ ]:
sd = ions.smeared_ions.density()

In [ ]:
sd = ions.smeared_ions[[0, 2]].density()

In [ ]:
sd = ions.smeared_ions[0].density

# Initialize system

In [ ]:
from envyron.physical import EnvironSystem

In [ ]:
system = EnvironSystem(
    ntypes=1,
    dim=0,
    axis=2,
    ions=ions,
)

In [ ]:
system.ntypes

In [ ]:
system.update()

In [ ]:
system.com

# Initialize electrons

In [ ]:
from envyron.physical import EnvironElectrons

In [ ]:
electrons = EnvironElectrons(grid)

In [ ]:
from dftpy.formats.cube import read_cube
cube = read_cube('density.cube')
electron_density = EnvironDensity(grid, cube[1])
electrons.update(electron_density)
electron_density.charge

In [ ]:
print(f"charge = {electrons.charge}")
print(f"count  = {electrons.count}")

# Initialize a core container

In [ ]:
from envyron.cores import CoreContainer

In [ ]:
cores = CoreContainer(
    label='outer',
    derivatives_core=fft_core,
    has_internal_correction=False,
)


# Initialize an electronic boundary

In [ ]:
from envyron.boundaries import ElectronicBoundary

In [ ]:
eboundary = ElectronicBoundary(
    rhomin=my_input.solvent.rhomin,
    rhomax=my_input.solvent.rhomax,
    electrons=electrons,
    mode='electronic',
    need_gradient=True,
    need_laplacian=True,
    need_hessian=True,
    deriv_method='chain',
    cores=cores,
    grid=grid,
    label='solvent'
)

In [ ]:
electrons.updating = True
eboundary.update()
electrons.updating = False

In [ ]:
print(f"volume  = {eboundary.volume}")
print(f"surface = {eboundary.surface}")

# Initialize an ionic boundary

In [ ]:
from envyron.boundaries import IonicBoundary

In [ ]:
iboundary = IonicBoundary(
    alpha=1.12,
    softness=0.5,
    ions=ions,
    mode='ionic',
    need_gradient=True,
    need_laplacian=True,
    need_hessian=True,
    deriv_method='lowmem',
    cores=cores,
    grid=grid,
    label='solvent'
)

In [ ]:
ions.updating = True
iboundary.update()
ions.updating = False

In [ ]:
print(f"volume  = {iboundary.volume}")
print(f"surface = {iboundary.surface}")

# Initialize a system boundary

In [ ]:
from envyron.boundaries import SystemBoundary

In [ ]:
sboundary = SystemBoundary(
    distance=my_input.solvent.distance,
    spread=my_input.solvent.spread,
    system=system,
    mode='system',
    need_gradient=True,
    need_laplacian=True,
    need_hessian=True,
    deriv_method='chain',
    cores=cores,
    grid=grid,
    label='solvent'
)

In [ ]:
system.updating = True
sboundary.update()
system.updating = False

In [ ]:
print(f"volume  = {sboundary.volume}")
print(f"surface = {sboundary.surface}")

# Initialize a dielectric

In [ ]:
from envyron.physical import EnvironDielectric

In [ ]:
dielectric = EnvironDielectric(sboundary, 100, False, True, False)

In [ ]:
import plotly.graph_objects as go

X, Y, Z = np.mgrid[0:grid.cell.diagonal()[0]:nr[0] * 1j,
                   0:grid.cell.diagonal()[1]:nr[1] * 1j,
                   0:grid.cell.diagonal()[2]:nr[2] * 1j]

fig = go.Figure(data=go.Volume(
    x=X.flatten(),
    y=Y.flatten(),
    z=Z.flatten(),
    value=sboundary.switch.flatten(),
    isomin=0.1,
    isomax=0.9,
    opacity=0.1,  # needs to be small to see through all surfaces
    surface_count=20,  # needs to be a large number for good volume rendering
))

fig.show()

# Read a New System from a Cube File

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
from envyron.io.cube import EnvironCube
water = EnvironCube()
water.load_cube('H2O.cube',units='Angstrom')

In [ ]:
import matplotlib.pyplot as plt
from matplotlib import ticker, cm
xx,yy,ff = water.cube2contour([6.8,7.,6.5],2)
fig, ax = plt.subplots()
cs = ax.contour(xx,yy,ff,locator=ticker.LogLocator(base=1.1),cmap=cm.PuBu_r)
ax.set_aspect('equal','box')
ax.set_xlabel('X (Angstrom)')
ax.set_ylabel('Y (Angstrom)')
cbar = fig.colorbar(cs)
plt.show()

In [ ]:
import matplotlib.pyplot as plt
x, y = water.cube2line([5.,7.,6.5],0)
plt.plot(x,y)
plt.show()

# Convert Cubefile Information for Environ Classes

In [ ]:
import numpy as np
natoms = len(water.atoms.numbers)
ntypes = len(np.unique(water.atoms.numbers))
ion_ids = list(np.unique(water.atoms.numbers))
ion_labels = list(np.unique(water.atoms.get_chemical_symbols()))
ion_weigths = list(np.unique(water.atoms.get_masses()))
itypes = [ ion_ids.index(id) for id in water.atoms.numbers]
zv = list(np.unique(water.atoms.get_initial_charges()))

In [ ]:
from envyron.domains import EnvironGrid
at = water.cell
nr = np.einsum('ijkl->jkl',water.grid).shape
grid = EnvironGrid(at, nr, label='system')

In [ ]:
from envyron.representations import EnvironDensity
rho = EnvironDensity(grid,water.data3D)
nelec = np.rint(rho.integral())

# Initialize a Setup

In [ ]:
from envyron.io.input import Input
my_input = Input(natoms=natoms, filename='volume.yml')

In [ ]:
from envyron.setup import Setup
my_setup = Setup(my_input)

In [ ]:
my_setup.init_cell(grid)

In [ ]:
my_setup.init_numerical(False)

# Initialize a Main

In [ ]:
from envyron.main import Main
environ = Main(my_setup,natoms,ntypes,itypes,zv,ion_ids)

In [ ]:
environ.update_cell_dependent_quantities()

In [ ]:
environ.update_ions(water.atoms.positions)

In [ ]:
environ.update_electrons(rho)

In [ ]:
environ.solvent.volume

# Initialize the Calculator

In [ ]:
from envyron.calculator import Calculator
my_calculator = Calculator(environ)

In [ ]:
my_calculator.energy()

In [ ]:
environ.evolume

In [ ]:
my_calculator.potential(True)

In [ ]:
environ.vsoftcavity.integral()/environ.setup.cell.volume